In [1]:
import Atlas
import pandas as pd

Atlas uses tape-based AD (C++'s XAD library). This will be used later for sensitivities.

In [2]:
try:
    tape = Atlas.Tape()
except:
    pass

#### Initialize market variables

Market variables are stored in the ```MarketStore```, we add the rate curves, rate indexes and fx exchange rates.

In [3]:
evalDate = Atlas.Date(1, Atlas.August, 2020)
store = Atlas.MarketStore(evalDate, Atlas.CLP()) # store with CLP as base currency

# define curve
curveDayCounter = Atlas.Actual360()
curveCompounding = Atlas.Simple
curveFrequency = Atlas.Annual

clpRate = Atlas.Dual(0.03)
usdRate = Atlas.Dual(0.01)
fx = Atlas.Dual(800)

# we need to register the input to the tape for later use
tape.registerInput(clpRate) 
tape.registerInput(usdRate) 
tape.registerInput(fx) 
tape.newRecording() # start recording, for later use


# add CLP Curve
strategy = Atlas.FlatForwardStrategy(evalDate, clpRate, curveDayCounter, curveCompounding, curveFrequency)
clpCurve = Atlas.YieldTermStructure(strategy)
index = Atlas.RateIndex(evalDate, curveFrequency, curveDayCounter, curveFrequency, curveCompounding)
store.addCurve("CLP", clpCurve, index)

# add USD Curve
strategy = Atlas.FlatForwardStrategy(evalDate, usdRate, curveDayCounter, curveCompounding, curveFrequency)
usdCurve = Atlas.YieldTermStructure(strategy)
store.addCurve("USD", usdCurve, index)

# add FX
store.addExchangeRate(Atlas.CLP(), Atlas.USD(), fx)


#### Create an instrument

We initialize the instrument with the corresponding discount curve id (passing the context parameter). As default, instrument coupon's are set as "local" currency, meaning that their current currency will match the store local currency.

In [4]:
#define interest rate
rateValue = Atlas.Dual(0.05)
dayCounter = Atlas.Thirty360()
compounding = Atlas.Simple
frequency = Atlas.Annual

rate = Atlas.InterestRate(rateValue, dayCounter, compounding, frequency)
discountContext = store.curveContext("CLP")
# define zero coupon instrument
notional = 100
startDate = evalDate
endDate = Atlas.Date(1, Atlas.August, 2025)
paymentFrequency = Atlas.Semiannual
instrument = Atlas.FixedRateBulletInstrument(startDate, endDate, paymentFrequency, notional, rate, discountContext)

#### Inspect the cashflows
We can use the CashflowProfiler visitor to check the instrument cashflows.

In [5]:
profiler = Atlas.CashflowProfiler()
profiler.visit(instrument)
interest = profiler.interests()
redemptions = profiler.redemptions()

df = pd.DataFrame({"Interest": interest, "Redemptions": redemptions})
df

,Interest,Redemptions
2021-2-1,2.5,NaN
2021-8-1,2.5,NaN
2022-2-1,2.5,NaN
2022-8-1,2.5,NaN
2023-2-1,2.5,NaN
2023-8-1,2.5,NaN
2024-2-1,2.5,NaN
2024-8-1,2.5,NaN
2025-2-1,2.5,NaN
2025-8-1,2.5,NaN


### Evaluation process

#### 1. Index the instrument
In the indexing phase, market variables are obtained and stored in a MarketRequest object. This information will be used by a Model, which is in charge of producing all the market data needed for later calculations.

In [6]:
indexer = Atlas.Indexer()
indexer.visit(instrument)
request = indexer.request()

#### 2. Setup a model and simulate market variables
Currently the only model available (SpotMarketDataModel), takes the market information and generates values assuming common linear product's assumptions.

In [7]:
model = Atlas.SpotMarketDataModel(request, store)
marketData = model.marketData(evalDate)

#### 3. Setup a visitor and evaluate
Visitor are the ones in charge to do evaluations. When visiting, Visitor will execute the precise code needed for each type of instrument.

- Instrument NPV: ```NPVCalculator```

This visitor calculates the NPV of each instruments and adds it to an internal variables called npv_, so if it visits many instrument, the value returned by ```results``` will be the sum of each NPV. In the case of a fixed bond, the NPV is being calculated as:

$$NPV^l = \frac{\Sigma_{1}^{N}c_{i}^{f}df^{f}_{i}}{fx^{f/l}}$$

In [8]:
npv = Atlas.Dual(0.0)
tape.registerOutput(npv)

npvCalculator = Atlas.NPVCalculator(marketData)
npvCalculator.visit(instrument)
npv = npvCalculator.results()
print("NPV: {:.4f}".format(Atlas.getValue(npv)))

NPV: 109.8990


If we want to calculate the insturment duration, we can use the tape (AD). In this case, the duration is being calculated as:

$$Dur = \frac{dNPV}{dr}$$

In [9]:
npv.setDerivative(1)
tape.computeAdjoints()
print("CLP Rate Sens: {:.4f}".format(clpRate.getDerivative()*0.0001))
print("USD Rate Sens: {:.4f}".format(usdRate.getDerivative()*0.0001))
print("Fx Rate Sens: {:.4f}".format(fx.getDerivative()))

CLP Rate Sens: -0.0439
USD Rate Sens: 0.0000
Fx Rate Sens: 0.0000


- Fixed Income Par Rate: ```ParSolver```

This visitor calculates the par rate of a given instrument (in this case, rates are not "accumulated" as before). The par rate is calculated, for a fixed rate instruments as follows:

$$r = \argmin_r (\frac{\Sigma_{1}^{T}c_{i}(r) df_{i}}{N} - df_0)^2$$

Where $df_0$ helps bringing the disbursement to the current evaluation date.

In [10]:
parSolver = Atlas.ParSolver(marketData)
parSolver.visit(instrument)
rate = parSolver.results()
print("Par Rate: {:.4f}%".format(Atlas.getValue(rate)*100))

Par Rate: 2.8579%


- Fixed Income Z-Spread: ```ZSpreadCalculator```

This visitor calculates the z-spread of a fixed rate instrument (does not apply to other types of instruments). 

$$s = \argmin_s ({\Sigma_{1}^{N}c_{i} df_{i}(s)} - NPV_{target})^2$$

Where $df_i$ will be calculated using the given day counter, compounding and frequency.

In [11]:
targetNPV = Atlas.Dual(100)
zspreadCalculator = Atlas.ZSpreadCalculator(marketData, targetNPV, curveDayCounter, curveCompounding, curveFrequency)
zspreadCalculator.visit(instrument)
zspread = zspreadCalculator.results()
print("Z-spread: CLP+{:.2f} bps".format(Atlas.getValue(zspread)*10000))

Z-spread: CLP+249.06 bps


#### Aside: Currency change

We now change the setup, we asume we are pricing a instrument in a different currency (USD) than the local one (CLP). In this case, we would expect the pricing system to convert all cashflows and sensitivites to local currency amounts.

In [12]:
instrument.currency(Atlas.USD())
instrument.discountCurveContext(store.curveContext("USD"))

indexer.clear()
indexer.visit(instrument)
request = indexer.request()

model = Atlas.SpotMarketDataModel(request, store) ## maybe add a setRequest method
marketData = model.marketData(evalDate)

# npv calculation
npvCalculator = Atlas.NPVCalculator(marketData) ## maybe add a setMarketData method
npvCalculator.visit(instrument)
npv = npvCalculator.results()
print("NPV: {:.4f}".format(Atlas.getValue(npv)))

NPV: 95598.9505


In [13]:
# adjoint calculation
npv.setDerivative(1)
tape.computeAdjoints()
print("CLP Rate Sens: {:.4f}".format(clpRate.getDerivative()))
print("USD Rate Sens: {:.4f}".format(usdRate.getDerivative()))
print("Fx Rate Sens: {:.4f}".format(fx.getDerivative()))

CLP Rate Sens: -439.2795
USD Rate Sens: -419628.2417
Fx Rate Sens: 119.4987


In [15]:
Atlas.Visitor()

TypeError: Atlas.Visitor: No constructor defined!